In [ ]:
import os
import re
import sys
import glob
import pickle
import tables
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib notebook

import tensorflow as tf
from tensorflow import keras

import dtw
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_areas, load_data_slide
from dlml.nn import predict

#### Find the best experiment given a set of tags

In [ ]:
area_ID = 1
area_measure = 'momentum'
stoch_load_bus_IDs = []
rec_bus_IDs = [3]
H_G1, D, DZA = None, None, None # 500, 2, 0
additional_tags = ['ReLU_none', 'converted_from_PowerFactory', 'all_stoch_loads', 'data_subset']
experiments = collect_experiments(area_ID, area_measure=area_measure, D=D, DZA=DZA, \
                                  stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                  rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                  verbose=True)
experiment_IDs = list(experiments.keys())
experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
MAPE = experiments[experiment_ID]['MAPE']
loss = experiments[experiment_ID]['loss']
val_loss = experiments[experiment_ID]['val_loss']
batch_loss = experiments[experiment_ID]['batch_loss']
tags = experiments[experiment_ID]['tags']
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

#### Load the model

In [ ]:
experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'parameters.pkl'), 'rb'))
epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
model = keras.models.load_model(best_checkpoint)
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
var_names = network_parameters['var_names']
print(f'Loaded network from {best_checkpoint}.')
print(f'Variable names: {var_names}')

#### Plot the model topology

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

#### Load the test set

In [ ]:
data_dirs = []
for area_ID,data_dir in zip(network_parameters['area_IDs'], network_parameters['data_dirs']):
    data_dirs.append(data_dir.format(area_ID))
data_dir = os.path.join('..', data_dirs[0])
data_files = sorted(glob.glob(data_dir + os.path.sep + '*_test_set.h5'))
X = []
for data_file in data_files:
    fid = tables.open_file(data_file)
    time = fid.root.time.read()
    x = [(fid.root[var_name].read()[:,:-1] - m) / s for var_name,m,s in zip(network_parameters['var_names'],
                                                                            x_train_mean,
                                                                            x_train_std)]
    X.append([np.reshape(y, [-1, 2400]) for y in x])
    fid.close()
dt = np.diff(time[:2])[0]
sampling_rate = 1 / dt

#### Predict the momentum using the model

In [ ]:
momentum = [np.squeeze(model.predict(x)) for x in X]
mean_momentum = [m.mean() for m in momentum]
print('Mean momentum:', mean_momentum)

#### Make as many submodels as there are layers

In [ ]:
N_layers = len(model.layers)
N_vars = len(var_names)
submodels = [
    keras.Model(inputs=model.inputs, outputs=[model.layers[j].output for j in range(i,i+N_vars)])
     for i in range(N_vars, N_layers - N_vars - 3, N_vars)
]
for layer in model.layers[-4:]:
    submodels.append(keras.Model(inputs=model.inputs, outputs=layer.output))

In [ ]:
i = 0
submodel = submodels[i]
submodel.summary()
y = [submodel.predict(x) for x in X]
print(y[0].shape)

In [ ]:
weights = np.squeeze(model.layers[1].weights[0].numpy())
fig,ax = plt.subplots(2, 8, figsize=(12, 3), sharex=True, sharey=True)
for i in range(2):
    for j in range(8):
        k = i * 8 + j
        ax[i][j].plot(weights[:,k], 'k', lw=1)

In [ ]:
plt.figure()
plt.plot(time[:2400], X[0][0][0,:], 'k', lw=1)
plt.plot(time[:2400], X[1][0][0,:], 'r', lw=1)

In [ ]:
plt.figure()
plt.plot(y[0][:n, :, 11].T, 'k', lw=1)
plt.plot(y[1][:n, :, 11].T, 'r', lw=1)

In [ ]:
plt.figure()
plt.plot(y[0][:n, :, 15].T, 'k', lw=1)
plt.plot(y[1][:n, :, 15].T, 'r', lw=1)

In [ ]:
rows, cols = y[0].shape[2] // 8, 8
fig,ax = plt.subplots(rows, cols, figsize=(cols*1.5, rows*1.5),
                      sharex=True, sharey=True)
n = 1
for i in range(rows):
    for j in range(cols):
        k = i * cols + j
        ax[i][j].plot(y[0][:n, :, k].T, 'k', lw=1)
        ax[i][j].plot(y[1][:n, :, k].T, 'r', lw=1)
#         ax[i][j].plot(y[0][:, :, k].mean(axis=0), 'k', lw=2)
#         ax[i][j].plot(y[1][:, :, k].mean(axis=0), 'r', lw=2)
        ax[i][j].set_xticks([0, y[0].shape[1]])
        for side in 'right','top':
            ax[i][j].spines[side].set_visible(False)
fig.tight_layout()

In [ ]:
raise Exception('Stop here')

## Continue from here

In [ ]:
N_blocks = len(data_sliding[0][var_names[0]])
inputs_to_network = []
inputs_to_classifier = []
outputs = []
for data_slid in data_sliding:
    for i in range(N_blocks):
        x = {var_name: tf.constant(data_slid[var_name][i:i+1,:], dtype=tf.float32) for var_name in var_names}
        inputs_to_network.append(np.concatenate([np.squeeze(data_slid[var_name][i:i+1,:]) for var_name in var_names]))
        inputs_to_classifier.append(np.squeeze(submodels[-3].predict(x)))
#         inputs_to_classifier.append(np.squeeze(submodels[-2].predict(x)))
        outputs.append(np.squeeze(submodels[-1].predict(x)))
inputs_to_network = np.array(inputs_to_network)
inputs_to_classifier = np.array(inputs_to_classifier)
outputs = np.array(outputs)
scaled_inputs_to_network = StandardScaler().fit_transform(inputs_to_network)
scaled_inputs_to_classifier = StandardScaler().fit_transform(inputs_to_classifier)

In [ ]:
area_inertia

In [ ]:
col = 'kr'
fig,ax = plt.subplots(2, 1, figsize=(10,5), sharex=False)
j = 10
for i in range(2):
    ax[0].plot(inputs_to_network[i*N_blocks + j], col[i], lw=1)
    ax[1].plot(inputs_to_classifier[i*N_blocks + j], col[i], lw=1, label=f'{outputs[i*N_blocks+j,0]:.2f}')
ax[1].legend(loc='best')

In [ ]:
inputs_to_classifier[i + j].sum()

In [ ]:
inputs_to_classifier[i*N_blocks + j].sum()

In [ ]:
dst_inputs_to_network = np.zeros(N_blocks)
dst_inputs_to_classifier = np.zeros(N_blocks)
for i in range(N_blocks):
    alignment = dtw.dtw(inputs_to_network[i], inputs_to_network[i+N_blocks], keep_internals=False)
    dst_inputs_to_network[i] = alignment.distance
    alignment = dtw.dtw(inputs_to_classifier[i], inputs_to_classifier[i+N_blocks], keep_internals=False)
    dst_inputs_to_classifier[i] = alignment.distance

In [ ]:
np.percentile(dst_inputs_to_network, [0.25,0.5,0.75])

In [ ]:
np.percentile(dst_inputs_to_classifier, [0.25,0.5,0.75])

In [ ]:
i = 0
m = np.min([inputs_to_network[i].min(), inputs_to_network[i+N_blocks].min()])
M = np.max([inputs_to_network[i].max(), inputs_to_network[i+N_blocks].max()])
dst_inputs_to_network = np.sqrt((inputs_to_network[i] - inputs_to_network[i+N_blocks]) ** 2) / (M - m)
m = np.min([inputs_to_classifier[i].min(), inputs_to_classifier[i+N_blocks].min()])
M = np.max([inputs_to_classifier[i].max(), inputs_to_classifier[i+N_blocks].max()])
dst_inputs_to_classifier = np.sqrt((inputs_to_classifier[i] - inputs_to_classifier[i+N_blocks]) ** 2) / (M - m)

plt.plot(dst_inputs_to_network, 'k', lw=0.5)
plt.plot(dst_inputs_to_classifier, 'r', lw=0.5)

In [ ]:
np.sqrt(np.sum((scaled_inputs_to_network[0] - scaled_inputs_to_network[N]) ** 2))

In [ ]:
np.sqrt(np.sum((scaled_inputs_to_classifier[0] - scaled_inputs_to_classifier[N]) ** 2))

In [ ]:
pca = PCA(n_components = 2)
pc_inputs_to_network = pca.fit_transform(scaled_inputs_to_network)
pc_inputs_to_classifier = pca.fit_transform(scaled_inputs_to_classifier)

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(7,4))
ax[0].plot(pc_inputs_to_network[:N,0], pc_inputs_to_network[:N,1], 'k.')
ax[0].plot(pc_inputs_to_network[N:,0], pc_inputs_to_network[N:,1], 'r.')
ax[1].plot(pc_inputs_to_classifier[:N,0], pc_inputs_to_classifier[:N,1], 'k.')
ax[1].plot(pc_inputs_to_classifier[N:,0], pc_inputs_to_classifier[N:,1], 'r.')

In [ ]:
embedding_inputs_to_network = umap.UMAP().fit_transform(scaled_inputs_to_network)
embedding_inputs_to_classifier = umap.UMAP().fit_transform(scaled_inputs_to_classifier)

In [ ]:
plt.plot(embedding_inputs_to_network[:N,0], embedding_inputs_to_network[:N,1],
         'ko', ms=4, markerfacecolor='w')
plt.plot(embedding_inputs_to_network[N:,0], embedding_inputs_to_network[N:,1],
         'ks', ms=4)
plt.plot(embedding_inputs_to_classifier[:N,0], embedding_inputs_to_classifier[:N,1],
         'ro', ms=4, markerfacecolor='w')
plt.plot(embedding_inputs_to_classifier[N:,0], embedding_inputs_to_classifier[N:,1],
         'rs', ms=4)

In [ ]:
output = outputs[0]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=True)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[1]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[2]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[3]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[4]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[5]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(10,3))
for i,a in enumerate(ax):
    a.plot(outputs[6+i][0], 'k', lw=1)
    a.set_xticks([])
    a.set_yticks([])
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(10,3))
for i,a in enumerate(ax):
    a.plot(outputs[6+i][0], 'k', lw=1)
    a.set_xticks([])
    a.set_yticks([])
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()